In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [5]:
# 1. 데이터셋 준비
df = pd.read_csv("seeds_dataset.txt", delim_whitespace=True, header=None)
df.columns = ["Area", "Perimeter", "Compactness", "KernelLength", "KernelWidth",
              "AsymmetryCoeff", "KernelGrooveLength", "Target"]
df = df.dropna()

df

Area  Perimeter  Compactness  KernelLength  KernelWidth  AsymmetryCoeff  \
0    15.26      14.84       0.8710         5.763        3.312           2.221   
1    14.88      14.57       0.8811         5.554        3.333           1.018   
2    14.29      14.09       0.9050         5.291        3.337           2.699   
3    13.84      13.94       0.8955         5.324        3.379           2.259   
4    16.14      14.99       0.9034         5.658        3.562           1.355   
..     ...        ...          ...           ...          ...             ...   
205  12.19      13.20       0.8783         5.137        2.981           3.631   
206  11.23      12.88       0.8511         5.140        2.795           4.325   
207  13.20      13.66       0.8883         5.236        3.232           8.315   
208  11.84      13.21       0.8521         5.175        2.836           3.598   
209  12.30      13.34       0.8684         5.243        2.974           5.637   

     KernelGrooveLength  Target  
0                 5.220       1  
1                 4.956       1  
2                 4.825       1  
3                 4.805       1  
4                 5.175       1  
..                  ...     ...  
205               4.870       3  
206               5.003       3  
207               5.056       3  
208               5.044       3  
209               5.063       3  

[210 rows x 8 columns]

In [7]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].astype(int).values - 1
y = to_categorical(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# 2. 모델 클래스 정의
import tensorflow as tf

class SeedClassifier(Model):
    def __init__(self):
        super(SeedClassifier, self).__init__()
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(32, activation='relu')
        self.output_layer = Dense(3, activation='softmax')

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)


In [11]:
# 3. 모델 인스턴스 생성
model = SeedClassifier()

# 4. 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 5. 학습
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)

# 6. 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {accuracy:.4f}")

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5203 - loss: 1.0690 - val_accuracy: 0.8529 - val_loss: 0.7869
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8659 - loss: 0.7523 - val_accuracy: 0.9118 - val_loss: 0.5181
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9155 - loss: 0.5056 - val_accuracy: 0.9412 - val_loss: 0.3741
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8536 - loss: 0.4711 - val_accuracy: 0.9118 - val_loss: 0.2950
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9011 - loss: 0.3433 - val_accuracy: 0.9118 - val_loss: 0.2460
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9204 - loss: 0.2877 - val_accuracy: 0.9118 - val_loss: 0.2143
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8867 - loss: 0.2582 - val_accuracy: 0.9118 - val_loss: 0.1887
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9203 - loss: 0.2412 - val_accuracy: 0.9118 - val_loss:

In [13]:
# 7. 예측 및 리포트 출력
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
print("\n📊 Classification Report:\n")
print(classification_report(y_true_classes, y_pred_classes))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

📊 Classification Report:

              precision    recall  f1-score   support

           0       0.83      0.91      0.87        11
           1       1.00      1.00      1.00        14
           2       0.94      0.88      0.91        17

    accuracy                           0.93        42
   macro avg       0.92      0.93      0.93        42
weighted avg       0.93      0.93      0.93        42

